In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
#Read all data.

main_puf = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/NTDB_2021/PUF_TRAUMA.csv", index_col = 'inc_key', low_memory = False)
preexisting_conditions = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/NTDB_2021/PUF_PREEXISTINGCONDITIONS_PIVOT.csv", index_col = 'Inc_Key', low_memory = False)
hospital_events = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/NTDB_2021/PUF_HOSPITALEVENTS_PIVOT.csv", index_col = 'Inc_Key', low_memory = False)
ais = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/NTDB_2021/PUF_AISDIAGNOSIS_PIVOT.csv", index_col = 'inc_key', low_memory = False)
ecode = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/NTDB_2021/PUF_ECODE.csv", index_col = 'inc_key', low_memory = False)
interventions = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/NTDB_2021/PUF_INTERVENTIONS.csv", index_col = 'Inc_Key', low_memory = False)

In [4]:
data = pd.concat([main_puf, preexisting_conditions, hospital_events, ais, ecode, interventions], axis=1)
data.columns = map(str.upper, data.columns)
data.shape

(1209097, 301)

#Inclusion and Exclusion Criteria

In [5]:
#Identify the patient population.

data = data[(data['AISSEVERITY1'] == 2) | (data['AISSEVERITY1'] == 3) | (data['AISSEVERITY1'] == 4) | (data['AISSEVERITY1'] == 5)]
data = data[(data['VERIFICATIONLEVEL'] == 1) | (data['VERIFICATIONLEVEL'] == 2)]

included = data.shape[0]

print('Number of patients included: ', included)

Number of patients included:  187547


In [6]:
#Exclude pediatric patients.

before = data.shape[0]
data = data[(data['AGEYEARS'] >= 18)]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  187547
Number of patients after exclusion:  163808
Number of patients excluded with this criteria:  23739


In [7]:
#Exclude AIS injury severity score ≥ 3 to neck.

before = data.shape[0]
data = data[data['AISSEVERITY3'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  163808
Number of patients after exclusion:  161420
Number of patients excluded with this criteria:  2388


In [8]:
#Exclude AIS injury severity score ≥ 3 to thorax.

before = data.shape[0]
data = data[data['AISSEVERITY4'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  161420
Number of patients after exclusion:  137497
Number of patients excluded with this criteria:  23923


In [9]:
#Exclude AIS injury severity score ≥ 3 to abdomen.

before = data.shape[0]
data = data[data['AISSEVERITY5'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  137497
Number of patients after exclusion:  135384
Number of patients excluded with this criteria:  2113


In [10]:
#Exclude AIS injury severity score ≥ 3 to spine.

before = data.shape[0]
data = data[data['AISSEVERITY6'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  135384
Number of patients after exclusion:  131222
Number of patients excluded with this criteria:  4162


In [11]:
#Exclude AIS injury severity score ≥ 3 to upper extremity.

before = data.shape[0]
data = data[data['AISSEVERITY7'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  131222
Number of patients after exclusion:  130507
Number of patients excluded with this criteria:  715


In [12]:
#Exclude AIS injury severity score ≥ 3 to lower extremity.

before = data.shape[0]
data = data[data['AISSEVERITY8'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  130507
Number of patients after exclusion:  125111
Number of patients excluded with this criteria:  5396


In [13]:
#Exclude AIS injury severity score ≥ 3 to unspecified body regions.

before = data.shape[0]
data = data[data['AISSEVERITY9'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  125111
Number of patients after exclusion:  125080
Number of patients excluded with this criteria:  31


In [14]:
#Exclude patients dead on arrival (SBP=0).

before = data.shape[0]
data = data[data['SBP'] != 0]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  125080
Number of patients after exclusion:  124334
Number of patients excluded with this criteria:  746


In [15]:
#Exclude patients with advanced directive limiting care.

before = data.shape[0]
data = data[data['CC_ADLC'] != 'Yes']
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  124334
Number of patients after exclusion:  119099
Number of patients excluded with this criteria:  5235


In [16]:
#Drop patients with invalid GCS.

before = data.shape[0]
data = data[data['PMGCSQ_VALID'] == 1]
after = data.shape[0]

excluded = before - after
print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  119099
Number of patients after exclusion:  92662
Number of patients excluded with this criteria:  26437


In [17]:
#Drop patients with unknown GCS.

before = data.shape[0]
data = data[data['TOTALGCS'].notna()]
after = data.shape[0]

excluded = before - after
print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  92662
Number of patients after exclusion:  89297
Number of patients excluded with this criteria:  3365


In [18]:
#Drop patients with unknown GCS-Motor.

before = data.shape[0]
data = data[data['GCSMOTOR'].notna()]
after = data.shape[0]

excluded = before - after
print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  89297
Number of patients after exclusion:  89179
Number of patients excluded with this criteria:  118


In [19]:
#Drop patients with unknown GCS-Verbal.

before = data.shape[0]
data = data[data['GCSVERBAL'].notna()]
after = data.shape[0]

excluded = before - after
print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  89179
Number of patients after exclusion:  89170
Number of patients excluded with this criteria:  9


In [20]:
#Drop patients with unknown GCS-Eye.

before = data.shape[0]
data = data[data['GCSEYE'].notna()]
after = data.shape[0]

excluded = before - after
print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  89170
Number of patients after exclusion:  89169
Number of patients excluded with this criteria:  1


In [21]:
#Drop patients with unknown pupillary response.

before = data.shape[0]
data = data[data['TBIPUPILLARYRESPONSE'].notna()]
after = data.shape[0]

excluded = before - after
print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  89169
Number of patients after exclusion:  86362
Number of patients excluded with this criteria:  2807


In [22]:
#Save data.

data.to_csv('/content/drive/MyDrive/TQP-MOST/2021.csv')